# Extracted spectra

## Notebook setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib qt5

In [3]:
import glob as glob
import matplotlib as mpl
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import numpy as np
import pandas as pd
import seaborn as sns

import bz2
import corner
import json
import pathlib
import pickle
import utils
import warnings

from astropy import constants as const
from astropy import units as uni
from astropy.io import ascii, fits
from astropy.time import Time
from mpl_toolkits.axes_grid1 import ImageGrid

# Default figure dimensions
FIG_WIDE = (11, 5)
FIG_LARGE = (8, 11)

# Figure style
sns.set(style="ticks", palette="colorblind", color_codes=True, context="talk")
params = utils.plot_params()
plt.rcParams.update(params)

## [Dowload data](https://www.dropbox.com/sh/f6rqfis0rdd0w1r/AAADJF0JI-YEx1cS9oCiwKP6a?dl=1)

Unzip this into a folder named `data` in the same level as this notebook

## Load

In [7]:
base_dir = "data/reduced/IMACS"

data_dict = {
#     "Transit 1": {
#         "data": utils.load_pickle(
#             f"{base_dir}/ut131219_a15_25_noflat/LCs_w50_bins.pkl"
#         ),
#         "comps_to_highlight": ["c15", "c18", "c21", "c23"],
#         "t0": Time("2013-12-19 03:22").jd,
#     },
    "Transit 1 (px)": {
        "data": utils.load_pickle(
            f"{base_dir}/ut131219/LCs_w50_bins_ut131219.pkl"
        ),
        "comps_to_highlight": ["c15", "c18", "c21", "c23"],
        "t0": Time("2013-12-19 03:22").jd,
    },
#     "Transit 2": {
#         "data": utils.load_pickle(
#             f"{base_dir}/ut150927_a15_25_noflat/LCs_w50_bins.pkl"
#         ),
#         "comps_to_highlight": ["c15", "c18", "c21", "c23"],
#         "t0": Time("2015-09-27 06:37").jd,
#     },
    "Transit 2 (px)": {
        "data": utils.load_pickle(
            f"{base_dir}/ut150927/LCs_w50_bins.pkl"
        ),
        "comps_to_highlight": ["c15", "c18", "c21", "c23"],
        "t0": Time("2015-09-27 06:37").jd,
    },
#     "Transit 3": {
#         "data": utils.load_pickle(
#             f"{base_dir}/ut161211_a24_42_noflat/LCs_w50_bins.pkl"
#         ),
#         "comps_to_highlight": ["c06", "c13"],
#         "t0": Time("2016-12-12 03:03").jd,
#     },
    "Transit 3 (px)": {
        "data": utils.load_pickle(
            f"{base_dir}/ut161211/LCs_w50_bins.pkl"
        ),
        "comps_to_highlight": ["c06", "c13"],
        "t0": Time("2016-12-12 03:03").jd,
    },
}

## Plot

In [19]:
for transit, transit_data in data_dict.items():
    spec = transit_data["data"]["spectra"]
    wavs = spec["wavelengths"]
    comp_names_to_use = transit_data["comps_to_highlight"]

    fig, ax = plt.subplots(figsize=FIG_WIDE)
    c = "darkgrey"
    for name, data in sorted(spec.items()):
        if "WASP50" in name:
            median_kwargs = {"c": "C8"}
        else:
            median_kwargs = None
            c = "grey"
        if (name in comp_names_to_use) or ("WASP50" in name):
            p, wav, flux = utils.plot_spec_file(
                ax,
                data = data,
                wavs = wavs,
                label = name,
                median_kwargs = median_kwargs,
                # fill_kwargs=fill_kwargs,
            )
    ax.legend(loc=1, fontsize=12)
    ax.set_title(transit)

    # Bins
    wavs_d, wavs_u = np.genfromtxt(f"{base_dir}/w50_bins.dat", unpack=True)

    if "Transit 1" in transit:
        wavs_d_1, wavs_u_1 = np.genfromtxt(f"{base_dir}/w50_bins_ut131219.dat", unpack=True)
        for i, (w_d, w_u) in enumerate(zip(wavs_d_1, wavs_u_1)):
            c = "k" if i % 2 == 0 else "darkgrey"
            ax.axvspan(w_d, w_u, alpha=0.25, color=c, lw=0)
    
    else:
        for i, (w_d, w_u) in enumerate(zip(wavs_d, wavs_u)):
            c = "k" if i % 2 == 0 else "darkgrey"
            ax.axvspan(w_d, w_u, alpha=0.25, color=c, lw=0)

    # Species
    species = {"Na I-D": 5892.9, "K I_avg": 7682.0, "Na I-8200_avg": 8189.0}
    [ax.axvline(wav, ls="--", lw=1, color="grey") for name, wav in species.items()]


    # Save
    title = transit
    ax.set_xlabel("Wavelength (Å)")
    ax.set_ylabel("Normalized Flux")
    # ax.set_title(title)
    ax.set_xlim(4_000, 10_000)
    title = title.lower().replace(" ", "_") + "_extr_spec"
    fig.tight_layout()
    fig.set_size_inches(FIG_WIDE)
    #utils.savefig(f"../paper/figures/extracted_spectra/{title}.pdf")